In [1]:
from models import *
from jepa import JEPA
from datasets import H5Dataset, SparseDataset
from trainer import Trainer

In [2]:
d1 = H5Dataset('train', 'multi')
d2 = H5Dataset('train', 'cite')

In [4]:
import pandas as pd

df1 = pd.read_hdf('data/train_cite_inputs.h5', start=1000, stop=2000)
df2 = pd.read_hdf('data/train_multi_targets.h5', start=1000, stop=2000)

rna1 = list(df1.keys())
rna2 = list(df2.keys())
df2.head()

gene_id,ENSG00000121410,ENSG00000268895,ENSG00000175899,ENSG00000245105,ENSG00000166535,ENSG00000256661,ENSG00000184389,ENSG00000128274,ENSG00000094914,ENSG00000081760,...,ENSG00000086827,ENSG00000174442,ENSG00000122952,ENSG00000198205,ENSG00000198455,ENSG00000070476,ENSG00000203995,ENSG00000162378,ENSG00000159840,ENSG00000074755
cell_id,,,,,,,,,,,,,,,,,,,,,
b31fa801888c,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,5.560683,...,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.0,0.000000,0.000000,0.000000
0e2c18dfbf7e,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.449378,4.449378,...,0.0,0.000000,0.000000,0.0,0.000000,5.136666,0.0,4.449378,0.000000,4.449378
352aba65831c,4.535933,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,4.535933,...,0.0,5.627375,4.535933,0.0,4.535933,4.535933,0.0,0.000000,0.000000,4.535933
53b116cbaf60,3.859019,0.0,0.0,0.0,0.0,0.0,0.0,0.0,4.541565,3.859019,...,0.0,0.000000,4.541565,0.0,0.000000,4.541565,0.0,0.000000,0.000000,0.000000
bb79553d92ea,4.181534,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.000000,0.000000,...,0.0,4.181534,4.867014,0.0,4.181534,0.000000,0.0,4.181534,4.181534,0.000000


In [5]:
for i in range(len(rna1)):
    rna1[i] = rna1[i].split('_')[0]

count = 0
idxs_shared = []
idxs_not_shared = []
for i, s in enumerate(rna1):
    if s in rna2:
        idxs_shared.append(i)
    else:
        idxs_not_shared.append(i)

In [6]:
model = LinearCoder([18386, 3664], dropout=0.01, input_2d=False, days=[2])
count_parameters(model)

80805856

In [15]:
batch_size = 128
multi = d1.get_dataloader(batch_size)
cite = d2.get_dataloader(batch_size)
model.train()
optim = torch.optim.Adam(model.parameters(), lr=0.04, weight_decay=0.0001)
for (x, day), y in multi:
    optim.zero_grad()
    inputs, targets = y[:, idxs_shared], y[:, idxs_not_shared]
    out = model(inputs, [torch.tensor(2),] * batch_size)
    
    loss = F.mse_loss(out, targets)
    loss.backward()
    optim.step()
    break